<a href="https://colab.research.google.com/github/AlvinKimata/Transformers/blob/main/rlhf_samples/Transformer_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q trl transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

In [4]:
#Get models.
model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
model_ref = create_reference_model(model)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

#Initialize trainer.
ppo_config = PPOConfig(
    batch_size = 1, 
    forward_batch_size = 1
)

#Encode a query.
query_txt = 'This morning I went to the '
query_tensor = tokenizer.encode(query_txt, return_tensors = 'pt')

In [7]:
#Get the model response.
response_tensor = respond_to_batch(model_ref, query_tensor)
print(response_tensor)

tensor([[ 1849, 15688, 24050,   284,  5725,   878,   314,  9153,   736,   284,
          7611,   553,   673,   531,    13,   366, 21979,   326,   373,  6600]])


In [10]:
decoded = [tokenizer.decode(x) for x in response_tensor]
print(decoded)

['\xa0sky sanctuary to consult before I headed back to campus," she said. "Cat that was eating']


In [14]:
#Create a ppo trainer.
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)

#Define a reward for this response.
reward = [torch.tensor(1.0)]

/usr/local/lib/python3.8/dist-packages/trl/trainer/ppo_trainer.py:194: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [15]:
#Train the model for one step.
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [16]:
train_stats

{'objective/kl': 0.0,
 'objective/kl_dist': [tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])],
 'objective/logprobs': [tensor([ -1.4446,  -8.7322,  -6.9775,  -2.0660,  -7.4861,  -7.2319,  -1.3399,
           -3.9275,  -2.1961,  -0.5052,  -5.8254,  -5.5736,  -1.8316,  -0.4258,
           -0.1459,  -0.5046, -11.6482,  -7.1312,  -2.0433,  -5.9775])],
 'objective/ref_logprobs': [tensor([ -1.4446,  -8.7322,  -6.9775,  -2.0660,  -7.4861,  -7.2319,  -1.3399,
           -3.9275,  -2.1961,  -0.5052,  -5.8254,  -5.5736,  -1.8316,  -0.4258,
           -0.1459,  -0.5046, -11.6482,  -7.1312,  -2.0433,  -5.9775])],
 'objective/kl_coef': 0.2,
 'objective/entropy': 83.01402282714844,
 'ppo/mean_non_score_reward': 0.0,
 'ppo/loss/policy': -0.10123981535434723,
 'ppo/loss/value': 33.112632751464844,
 'ppo/loss/total': 3.2100234031677246,
 'ppo/policy/entropy': 4.176176071166992,
 'ppo/policy/approxkl': 0.07178714871406555,
 'ppo/policy/policykl': 0.00272612553089